# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [3]:
from zipfile import ZipFile
zip = ZipFile('News_Category_Dataset_v2.zip')
zip.extractall()
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [4]:
df['combined_text'] = df['headline'] + ' ' +  df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [5]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [6]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [7]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55588352, 67376200)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [8]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [9]:
wv.most_similar("Texas")

[('Pennsylvania', 0.7981922030448914),
 ('Oklahoma', 0.7922916412353516),
 ('Maryland', 0.7910659909248352),
 ('Louisiana', 0.7901260852813721),
 ('Georgia', 0.7897605895996094),
 ('Ohio', 0.7848389744758606),
 ('Florida', 0.7754758596420288),
 ('Arkansas', 0.7741115093231201),
 ('Illinois', 0.7734614610671997),
 ('California', 0.7718496918678284)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [10]:
wv.most_similar(negative='Texas')

[('much-vaunted', 0.4667600989341736),
 ("'tough", 0.43998974561691284),
 ('Profit-Minded', 0.3980049192905426),
 ('maitre', 0.36873379349708557),
 ("'Banned", 0.3638817071914673),
 ('Revolve', 0.3600705862045288),
 ('Dart', 0.3566845655441284),
 ('Centcom', 0.35488495230674744),
 ('Likened', 0.3546619415283203),
 ('Jobseekers', 0.35089176893234253)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [11]:
wv['Texas']

array([ 0.20116344, -0.40157595,  1.1847062 , -1.1255125 , -0.43515155,
       -1.3442647 , -1.6469452 , -0.06276993,  0.573681  ,  0.24603185,
        1.2723798 ,  1.1292816 ,  0.28008676, -1.05502   , -0.889999  ,
       -1.3182635 ,  0.7742109 , -1.2959968 ,  0.18755351, -1.4739766 ,
       -1.2418128 ,  0.6456338 , -1.4719254 ,  1.6410166 ,  1.4075512 ,
       -1.0142179 ,  0.9190244 ,  1.0541708 , -0.36947766, -1.1985301 ,
        3.2353482 ,  0.5657587 ,  0.8361077 ,  0.73843306, -0.5957873 ,
        0.50160754, -0.12266119,  0.96507376,  0.57393306, -0.42340112,
        0.6935443 , -0.01830146, -0.10149425,  1.2692238 ,  1.0839183 ,
        1.4689301 , -2.0105634 , -0.1029269 , -2.6337664 , -3.7438378 ,
       -1.6254752 , -3.088566  , -1.0244648 ,  0.7270971 , -0.7508847 ,
        0.757342  ,  2.103466  ,  2.5731978 ,  0.01107936,  1.7389283 ,
        1.285745  , -1.64723   , -2.2073894 ,  0.24645858, -0.659496  ,
        0.6662499 ,  0.0711427 , -1.798904  , -0.7583374 ,  0.06

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [12]:
wv.vectors

array([[ 1.00010669e+00, -8.78780246e-01,  1.10324764e+00, ...,
         2.00073409e+00,  1.03300619e+00, -1.78016067e-01],
       [ 2.48364353e+00, -7.73303151e-01,  1.26027870e+00, ...,
        -4.93194789e-01,  4.02111977e-01,  7.76964784e-01],
       [ 1.16664815e+00, -1.87439930e+00,  2.52087688e+00, ...,
         8.31537306e-01,  1.43367672e+00, -9.31001231e-02],
       ...,
       [ 3.11481170e-02,  2.51867138e-02,  7.27189751e-03, ...,
         8.75571296e-02, -1.59825757e-02, -2.20557395e-02],
       [ 1.02539934e-01,  9.63269640e-03,  1.85766723e-02, ...,
         4.68888413e-03,  6.61340356e-02,  4.30064350e-02],
       [-1.32009918e-02, -1.47269070e-02, -3.17909196e-02, ...,
        -5.34008071e-02, -2.46918411e-03, -1.11347279e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [14]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('tch', 0.5571240782737732),
 ('BULLSH', 0.5532172918319702),
 ('cked', 0.5507582426071167),
 ('ke', 0.527708888053894),
 ('cking', 0.5256286859512329),
 ('ck', 0.5235520601272583),
 ('NSYNC', 0.5227694511413574),
 ('Place-based', 0.5175246596336365),
 ('ssy', 0.5140359401702881),
 ('NSYNC-Inspired', 0.5076217651367188)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!